### Set Up for Project Imports


In [17]:
import sys
from pathlib import Path

In [3]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [4]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from data.api import UcIrvineAPI, BureauEconomicAnalysisAPI
from data.dataset_id import DatasetIDs
import json
import pandas

In [15]:
pandas.set_option('display.max_colwidth', None) # show all text in cells

## Data

In [13]:
apartment_for_rent_classified = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

/Users/isaacpacheco/Documents/School/Graduate/Fall 2025/CS 577 - Data Science/Project/Github/.venv/lib/python3.13/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [14]:
print(json.dumps(apartment_for_rent_classified.metadata, indent=2))

{
  "uci_id": 555,
  "name": "Apartment for Rent Classified",
  "repository_url": "https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified",
  "data_url": "https://archive.ics.uci.edu/static/public/555/data.csv",
  "abstract": "This is a dataset of classified for apartments for rent in USA.\n",
  "area": "Business",
  "tasks": [
    "Classification",
    "Regression",
    "Clustering"
  ],
  "characteristics": [
    "Multivariate"
  ],
  "num_instances": 10000,
  "num_features": 21,
  "feature_types": [
    "Categorical",
    "Integer"
  ],
  "demographics": [],
  "target_col": null,
  "index_col": [
    "id"
  ],
  "has_missing_values": "no",
  "missing_values_symbol": null,
  "year_of_dataset_creation": 2019,
  "last_updated": "Mon Feb 26 2024",
  "dataset_doi": "10.24432/C5X623",
  "creators": [],
  "intro_paper": null,
  "additional_info": {
    "summary": "The dataset contains of 10'000 or 100'000 rows and of 22 columns The data has been cleaned in the way that \r\nc

In [11]:
data_set_list = BureauEconomicAnalysisAPI.get_data_set_list()
print(data_set_list)

                DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyIndustry                         

In [16]:
parameter_list = BureauEconomicAnalysisAPI.get_parameter_list('Regional')
print(parameter_list)

  ParameterName ParameterDataType  \
0       GeoFips            string   
1      LineCode           integer   
2     TableName            string   
3          Year            string   

                                                                                                                                                                                                                                                                                                                                    ParameterDescription  \
0  Comma-delimited list of 5-character geographic codes; COUNTY for all counties, STATE for all states, MSA for all MSAs, MIC for all Micropolitan Areas, PORT for all state metro/nonmetro portions, DIV for all Metropolitan Divisions, CSA for all Combined Statistical Areas, state post office abbreviation for all counties in one state (e.g. NY)   
1                                                                                                                     